# Pre-Production Context Engine

Copyright 2025, Denis Rothman

*From Prototype to pre-production: Hardening the Context Engine*

This notebook represents a significant architectural evolution from a monolithic prototype to a modular, pre-production system. It no longer contains the core logic itself but instead acts as a high-level "Control Deck" for a powerful multi-agent AI.

The engine's components—utility functions, helper classes, specialized agents, the agent registry, and the core planner/executor—have been refactored into a reusable commons library. These modules are dynamically downloaded from a central GitHub repository, ensuring that this notebook always runs the latest, most stable version of the engine.

The workflow is simple:

**Initialize**: Download the library and set up API clients.

**Configure**: Define all parameters (models, namespaces) in a single config dictionary.

**Execute**: Provide a high-level goal and run the engine.

This modular design promotes maintainability, reusability, and scalability, abstracting away the underlying complexity and allowing the user to focus purely on the task at hand. The output provides both the final generated content and a detailed technical trace for developers.


# I. Inititalization

# GitHub

In [1]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/registry.py.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    14    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 404
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    14    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 404
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0    14    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (22) The requested URL returned error: 404
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent

In [2]:
# This cell will be deleted when repo goes public
import glob

# glob.glob("/content/*.py") will return a list of matching files.
# If the list is not empty, it evaluates to True.
# If the list is empty (no .py files found), it evaluates to False.

if glob.glob("/content/*.py"):
    filesdownloaded = True
else:
    filesdownloaded = False

print(f"Are .py files present in /content? {filesdownloaded}")

import os
import glob
from google.colab import userdata

# --- Configuration ---
OWNER = "Denis2054"
REPO = "Context-Engineering"
BRANCH = "main"

FILES_TO_DOWNLOAD = [
    {"remote": "commons/utils.py", "local": "utils.py"},
    {"remote": "commons/helpers.py", "local": "helpers.py"},
    {"remote": "commons/agents.py", "local": "agents.py"},
    {"remote": "commons/registry.py", "local": "registry.py"},
    {"remote": "commons/engine.py", "local": "engine.py"}
]
# --- End Configuration ---
# Check if all target files already exist
all_files_exist = all(os.path.exists(f["local"]) for f in FILES_TO_DOWNLOAD)

if not all_files_exist:
    print("ℹ️ Files not found. Attempting download using private token...")

    try:
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            print("🛑 Error: 'GITHUB_TOKEN' not found in Colab Secrets.")
        else:
            # Loop and download each file that is missing
            for f in FILES_TO_DOWNLOAD:
                if not os.path.exists(f["local"]):
                    # --- FIX: Corrected the typo 'httpsfs://' to 'https://' ---
                    api_url = f"https://api.github.com/repos/{OWNER}/{REPO}/contents/{f['remote']}?ref={BRANCH}"

                    # This is the !curl command, run from Python
                    # We use os.system to execute it
                    print(f"Downloading {f['local']}...")
                    command = (
                        f"curl -Lfs "
                        f"-H \"Authorization: Bearer {token}\" "
                        f"-H \"Accept: application/vnd.github.v3.raw\" "
                        f"\"{api_url}\" -o \"{f['local']}\""
                    )

                    # os.system returns 0 on success
                    if os.system(command) != 0:
                        print(f"🛑 Error downloading {f['local']}.")

            # Final check
            if all(os.path.exists(f["local"]) for f in FILES_TO_DOWNLOAD):
                print("✅ All files successfully downloaded.")
            else:
                print("🛑 Some files failed to download.")

    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("✅ Files already present. Skipping download.")



Are .py files present in /content? False
ℹ️ Files not found. Attempting download using private token...
✅ All files successfully downloaded.


# Installation and client setup

In [3]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [4]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [5]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

def execute_and_display(goal, config, client, pc):
    """
    Runs the context engine with a given goal and configuration,
    then displays the final output and the technical trace.
    """
    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config  # Unpack the config dictionary into keyword arguments
    )

    # 2. Display the Final Result for the main reader
    print("--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace for the developer/technical reader
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # Use pprint for a clean, readable dictionary output
        pp = pprint.PrettyPrinter(indent=2)
        pp.pprint(trace.steps)

## Control Deck configuration

In [6]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#II.CONTROL DECK

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [7]:
#Example 1
# Define the high-level goal
goal = "Write a short, suspenseful scene for a children's story about the Apollo 11 moon landing, highlighting the danger."
# Call the execution function from the cell above
execute_and_display(goal, config, client, pc)

--- FINAL OUTPUT ---


July 20, 1969. Eagle slides down into black and gray. I watch the shadows move. Buzz breathes beside me.

Beep. Beep. 1201. Then 1202. The screen glares. The cabin hums. My throat tightens. Mission Control cuts in. Calm. You’re go.

Fuel thins. About a minute left. The clock ticks loud in my head. The Moon waits. Still. Cold. Close.

West Crater ahead. Rocks like teeth. I take the stick. Manual now. Light touch. Damp palm. I swing us past the boulders. Searching for smooth ground. The unknown below presses up.

The engine thrums through the frame. Dust boils. It climbs in sheets. It hides the surface. Shadows smear. The world turns hazy, brown-gray, strange.

Buzz calls it out. Height. Speed. Steady voice. A clean metronome in the murk. I listen. I ease. I nudge.

Silence crowds in. Breath small. Time thin. Legs reaching. We hover over ghosted ground. Hold it. Hold it.

A soft settle. A stop.

I key the mike. Tranquility Base here, the Eagle has landed.

The threat slips back into the shadows. Alarms quiet. Dust falls. We breathe again. Smart choices. Steady teamwork. Calm hands. First feet, soon, on a new world.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 126.20 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Increase tension and create '
                                  'suspense.", "style_guide": "Use short, '
                                  'sharp sentences. Focus on sensory details '
                                  '(sounds, shadows). Maintain a slightly '
                                  'eerie but age-appropriate tone.", '
                                  '"participants": [{"role": "Agent", '
                                  '"description": "The protagonist '
                                  'experiencing the events."}, {"role": '
                                  '"Source_of_Threat", "description": "The '
                                  'underlying danger or mystery."}], '
                                  '"instruction": "Rewrite the provided facts '
                              